In [18]:
import pandas as pd
import numpy as np
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import torch

# Download NLTK data files (only need to run once)
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

# Apply tqdm to all .apply() functions by using progress_apply
tqdm.pandas()

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\97798\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\97798\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\97798\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
openmoji_df = pd.read_csv('../data/openmoji.csv')
emojipedia_df = pd.read_csv('../data/emojipedia.csv')
llm_df = pd.read_parquet('../data/llmemoji.parquet')

In [20]:
# Convert Unicode string (e.g., 'U+1F600', 'U+263A,FE0F') to hex code ('1F600', '263A-FE0F').

def unicode_to_hex(unicode_str):
    unicode_str = unicode_str.replace(",", " ")  # Replace commas with spaces
    # First splitting the input string to a list of substrings
    # Loops though each substring
    # Removes the U+ prefix from each substring
    hex_values = [u.replace("U+", "") for u in unicode_str.split()]
    # Join the values with hyphens
    return "-".join(hex_values)

# Convert 'unicode' column in emojipedia_df and llm_df to 'hexcode'
emojipedia_df['hexcode'] = emojipedia_df['Codepoints Hex'].progress_apply(unicode_to_hex)
llm_df['hexcode'] = llm_df['unicode'].progress_apply(unicode_to_hex)

100%|██████████| 5034/5034 [00:00<00:00, 715345.11it/s]


In [21]:
# Making the hexcode uniform in all 3 dataframes for merging
# removing -f30f (differentiation between image type emoji and textual type emoji)
# removing -200d (differentiation for emoji with skin-tone)

openmoji_df['hexcode'] = openmoji_df['hexcode'].str.replace('-FE0F', '', regex=True)
openmoji_df['hexcode'] = openmoji_df['hexcode'].str.replace('-200D', '', regex=True)
llm_df['hexcode'] = llm_df['hexcode'].str.replace('-FE0F', '', regex=True)
llm_df['hexcode'] = llm_df['hexcode'].str.replace('-200D', '', regex=True)
emojipedia_df['hexcode'] = emojipedia_df['hexcode'].str.replace('-FE0F', '', regex=True)
emojipedia_df['hexcode'] = emojipedia_df['hexcode'].str.replace('-200D', '', regex=True)

In [22]:
# Checking for duplicates
duplicate_counts = openmoji_df['hexcode'].value_counts()
duplicates = duplicate_counts[duplicate_counts > 1]
print(duplicates)

hexcode
1F3F3    2
Name: count, dtype: int64


In [23]:
# Checking for duplicates
duplicate_counts = llm_df['hexcode'].value_counts()
duplicates = duplicate_counts[duplicate_counts > 1]
print(duplicates)

hexcode
1F3C3-1F3FD-2640-27A1           4
1F6B6-1F3FF-2642-27A1           4
1F6B6-1F3FF-2640-27A1           4
1F6B6-1F3FE-2640-27A1           4
1F6B6-1F3FC-2640-27A1           4
                               ..
1F9D1-1F3FC-2764-1F9D1-1F3FD    2
1F469-1F3FF-2764-1F468-1F3FD    2
1F9D1-1F3FD-2764-1F9D1-1F3FF    2
1F327                           2
2601                            2
Name: count, Length: 1160, dtype: int64


In [24]:
duplicate_counts = emojipedia_df['hexcode'].value_counts()
duplicates = duplicate_counts[duplicate_counts > 1]
print(duplicates)

Series([], Name: count, dtype: int64)


In [25]:
# Removing duplicates
openmoji_df = openmoji_df[~openmoji_df.duplicated(subset=['hexcode'], keep=False)]
llm_df = llm_df[~llm_df.duplicated(subset=['hexcode'], keep=False)]

In [26]:
# Merge the dataframes on 'hexcode'
merged_df = openmoji_df.merge(emojipedia_df, on='hexcode', how='outer')
merged_df = merged_df.merge(llm_df, on='hexcode', how='outer')
# Convert hexcode to lowercase
merged_df['hexcode'] = merged_df['hexcode'].str.lower()

In [27]:
merged_df.head()

,emoji,hexcode,group,subgroups,annotation,tags_x,openmoji_tags,openmoji_author,openmoji_date,skintone,...,Title,DescribedBy,URL,Description,Codepoints Hex,character,unicode_y,short description,tags_y,LLM description
0,#️⃣,0023-20e3,symbols,keycap,keycap: #,keycap,NaN,Selina Lange,2019-05-06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,*️⃣,002a-20e3,symbols,keycap,keycap: *,keycap,NaN,Selina Lange,2019-05-06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-,002d,extras-unicode,symbol-other,hyphen-minus,NaN,"hyphen, minus, dash, line",Robert Winslow,2022-12-24,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0️⃣,0030-20e3,symbols,keycap,keycap: 0,keycap,NaN,Selina Lange,2019-05-06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1️⃣,0031-20e3,symbols,keycap,keycap: 1,keycap,NaN,Selina Lange,2019-05-06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4298 entries, 0 to 4297
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   emoji                  4282 non-null   object 
 1   hexcode                4298 non-null   object 
 2   group                  4282 non-null   object 
 3   subgroups              4282 non-null   object 
 4   annotation             4282 non-null   object 
 5   tags_x                 1906 non-null   object 
 6   openmoji_tags          391 non-null    object 
 7   openmoji_author        4282 non-null   object 
 8   openmoji_date          4282 non-null   object 
 9   skintone               1875 non-null   object 
 10  skintone_combination   2198 non-null   object 
 11  skintone_base_emoji    2198 non-null   object 
 12  skintone_base_hexcode  2198 non-null   object 
 13  unicode_x              3902 non-null   object 
 14  order                  3781 non-null   float64
 15  Grou

In [29]:
# Select and rename required columns
final_df = merged_df.copy()
final_df = final_df[['emoji', 'hexcode', 'annotation', 'tags_x', 'openmoji_tags', 'Description', 'tags_y', 'LLM description']]
final_df.columns = [
    'emoji', 'hexcode', 'openmoji_annotation', 'openmoji_tags_1', 'openmoji_tags_2', 'emojipedia_description', 'llm_tags', 'llm_description'
]

In [30]:
final_df.head()

,emoji,hexcode,openmoji_annotation,openmoji_tags_1,openmoji_tags_2,emojipedia_description,llm_tags,llm_description
0,#️⃣,0023-20e3,keycap: #,keycap,NaN,NaN,NaN,NaN
1,*️⃣,002a-20e3,keycap: *,keycap,NaN,NaN,NaN,NaN
2,-,002d,hyphen-minus,NaN,"hyphen, minus, dash, line",NaN,NaN,NaN
3,0️⃣,0030-20e3,keycap: 0,keycap,NaN,NaN,NaN,NaN
4,1️⃣,0031-20e3,keycap: 1,keycap,NaN,NaN,NaN,NaN


## Handling Tags and Descriptions

In [31]:
def clean_text(text):
    if not isinstance(text, str) or pd.isna(text) or text.strip().lower() == "nan":  
        return ""  # Return empty string for NaN or "nan" strings
    text = text.lower().strip() # Convert to lowercase and remove unnecessary spaces
    # Keep only letters, numbers, spaces, * and #
    text = re.sub(r'[^a-z0-9\s*#]', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stop words
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    # Lemmatize the tokens
    # lemmatizer = WordNetLemmatizer()
    # tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ', '.join(tokens)

def remove_duplicates(text):
    words = [word.strip() for word in text.split(",")]  # Split by commas and strip spaces
    unique_words = sorted(set(word.strip() for word in text.split(",")))
    return ', '.join(unique_words)  # Join back into a string

In [32]:
# Clean openmoji_annotation column
final_df["cleaned_annotations"] = final_df["openmoji_annotation"].progress_apply(clean_text)
# Clean llm_tags
final_df["cleaned_llm_tags"] = final_df["llm_tags"].progress_apply(clean_text)

# List of columns to merge
columns_to_merge = ["cleaned_annotations", "openmoji_tags_1", "openmoji_tags_2", "cleaned_llm_tags"]

# Fill NaN with empty strings, then merge columns
final_df["merged_tags"] = final_df[columns_to_merge].fillna("").agg(
    lambda x: ", ".join(filter(None, map(str, x))), axis=1
)

  0%|          | 0/4298 [00:00<?, ?it/s]

100%|██████████| 4298/4298 [00:00<00:00, 1370570.87it/s]


In [33]:
# Remove duplicates
final_df["final_tags"] = final_df["merged_tags"].progress_apply(remove_duplicates)

100%|██████████| 4298/4298 [00:00<00:00, 272873.56it/s]


In [34]:
# Limiting only 2 sentences because the content is too long with too many references to other emoji, which might be more confusing later for the model to learn.

"""
Reference: Chat-GPT-4o
Prompt: My df_images has a emojipedia_description column. Only keep the first two sentences in the column and remove the rest.
"""
# Function to keep only the first two sentences
def keep_first_two_sentences(description):
    if pd.isna(description):  # Handle missing values (NaNs)
        return description
    sentences = re.split(r'(?<=[.!?]) +', description.strip())  # Split by sentence-ending punctuation (., !, ?)
    return ' '.join(sentences[:2])  # Return only the first two sentences

# Apply the function to the 'llm_description' column
final_df['emojipedia_description'] = final_df['emojipedia_description'].progress_apply(keep_first_two_sentences)

  0%|          | 0/4298 [00:00<?, ?it/s]

100%|██████████| 4298/4298 [00:00<00:00, 285528.36it/s]


In [35]:
final_df.head()

,emoji,hexcode,openmoji_annotation,openmoji_tags_1,openmoji_tags_2,emojipedia_description,llm_tags,llm_description,cleaned_annotations,cleaned_llm_tags,merged_tags,final_tags
0,#️⃣,0023-20e3,keycap: #,keycap,NaN,NaN,NaN,NaN,"keycap, #",,"keycap, #, keycap","#, keycap"
1,*️⃣,002a-20e3,keycap: *,keycap,NaN,NaN,NaN,NaN,"keycap, *",,"keycap, *, keycap","*, keycap"
2,-,002d,hyphen-minus,NaN,"hyphen, minus, dash, line",NaN,NaN,NaN,hyphenminus,,"hyphenminus, hyphen, minus, dash, line","dash, hyphen, hyphenminus, line, minus"
3,0️⃣,0030-20e3,keycap: 0,keycap,NaN,NaN,NaN,NaN,"keycap, 0",,"keycap, 0, keycap","0, keycap"
4,1️⃣,0031-20e3,keycap: 1,keycap,NaN,NaN,NaN,NaN,"keycap, 1",,"keycap, 1, keycap","1, keycap"


In [36]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4298 entries, 0 to 4297
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   emoji                   4282 non-null   object
 1   hexcode                 4298 non-null   object
 2   openmoji_annotation     4282 non-null   object
 3   openmoji_tags_1         1906 non-null   object
 4   openmoji_tags_2         391 non-null    object
 5   emojipedia_description  1885 non-null   object
 6   llm_tags                2622 non-null   object
 7   llm_description         2622 non-null   object
 8   cleaned_annotations     4298 non-null   object
 9   cleaned_llm_tags        4298 non-null   object
 10  merged_tags             4298 non-null   object
 11  final_tags              4298 non-null   object
dtypes: object(12)
memory usage: 403.1+ KB


## Embedding using Sentence BERT

In [37]:
# Use GPU if available
"""
Reference: https://pytorch.org/get-started/locally/
"""

# Check for NVIDIA GPU
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use CUDA (NVIDIA GPU)
    print("Using NVIDIA GPU (CUDA)")

# Check for Mac Silicon GPU (MPS)
elif torch.backends.mps.is_available():
    device = torch.device("mps")  # Use Metal Performance Shaders (Mac Silicon GPU)
    print("Using Mac GPU (MPS)")

# Default to CPU if no GPU is available
else:
    device = torch.device("cpu")
    print("Using CPU")

Using CPU


In [38]:
# Load SBERT model
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')
sbert_model = sbert_model.to(device)

# Ensure text columns are strings
# final_df["emojipedia_description"] = final_df["emojipedia_description"].fillna("").astype(str)
# final_df["llm_description"] = final_df["llm_description"].fillna("").astype(str)

# Return SBERT embedding for a given text.
def embed_text(text):
    if pd.isna(text) or text.strip() == "":
        return np.zeros(384)  # Return zero vector for missing values (SBERT output size = 384)
    return sbert_model.encode(text)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [39]:
"""
Reference: https://sbert.net/
"""

# Apply SBERT embeddings to each columns
print("Embedding final_tags...")
final_df["tags_embedding"] = final_df["final_tags"].progress_apply(embed_text)

print("Embedding emojipedia_description...")
final_df["emojipedia_embedding"] = final_df["emojipedia_description"].progress_apply(embed_text)

print("Embedding llm_description...")
final_df["llm_embedding"] = final_df["llm_description"].progress_apply(embed_text)

Embedding final_tags...


100%|██████████| 4298/4298 [00:48<00:00, 88.30it/s] 


Embedding emojipedia_description...


100%|██████████| 4298/4298 [00:25<00:00, 166.59it/s]


Embedding llm_description...


100%|██████████| 4298/4298 [00:37<00:00, 113.64it/s]


In [40]:
final_df["combined_embedding"] = final_df.apply(
    lambda row: np.concatenate([row["tags_embedding"], row["emojipedia_embedding"], row["llm_embedding"]]), axis=1
)

In [41]:
final_df.head()

,emoji,hexcode,openmoji_annotation,openmoji_tags_1,openmoji_tags_2,emojipedia_description,llm_tags,llm_description,cleaned_annotations,cleaned_llm_tags,merged_tags,final_tags,tags_embedding,emojipedia_embedding,llm_embedding,combined_embedding
0,#️⃣,0023-20e3,keycap: #,keycap,NaN,NaN,NaN,NaN,"keycap, #",,"keycap, #, keycap","#, keycap","[0.0251947, -0.049892798, 0.049407776, -0.0125...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.025194700807332993, -0.04989279806613922, 0..."
1,*️⃣,002a-20e3,keycap: *,keycap,NaN,NaN,NaN,NaN,"keycap, *",,"keycap, *, keycap","*, keycap","[0.030324776, -0.047731027, 0.04585725, -0.008...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.030324775725603104, -0.047731027007102966, ..."
2,-,002d,hyphen-minus,NaN,"hyphen, minus, dash, line",NaN,NaN,NaN,hyphenminus,,"hyphenminus, hyphen, minus, dash, line","dash, hyphen, hyphenminus, line, minus","[-0.024587186, 0.10079277, 0.011220684, -0.040...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.02458718605339527, 0.1007927730679512, 0.0..."
3,0️⃣,0030-20e3,keycap: 0,keycap,NaN,NaN,NaN,NaN,"keycap, 0",,"keycap, 0, keycap","0, keycap","[0.043693528, -0.016325166, -0.030014966, 0.02...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.043693527579307556, -0.016325166448950768, ..."
4,1️⃣,0031-20e3,keycap: 1,keycap,NaN,NaN,NaN,NaN,"keycap, 1",,"keycap, 1, keycap","1, keycap","[0.02213429, -0.073349945, -0.005648202, -0.00...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.022134289145469666, -0.07334994524717331, -..."


## Linking Images

In [42]:
# Define paths
processed_image_path = "../data/tensor_images/"
brands = ["GoogleEmoji", "JoyPixelsEmoji", "OpenMojiEmoji", "TwitterEmoji"]

# Function to get image paths for a given hexcode
def get_image_paths(hexcode):
    image_paths = {}

    for brand in brands:
        brand_path = os.path.join(processed_image_path, brand)
        if not os.path.exists(brand_path):
            continue  # Skip if folder doesn't exist

        # List all files in the category
        matching_files = [f for f in os.listdir(brand_path) if hexcode in f]  # Match anywhere in the filename
        
        if matching_files:
            # Take the first match (if multiple exist)
            image_paths[brand] = os.path.join(brand_path, matching_files[0])
        else:
            image_paths[brand] = None  # No matching file found

    return image_paths

df_images = final_df.copy()  # Work on a copy to be safe

# Apply the function to get image paths for each row
df_images[['google_image_path', 'joypixels_image_path', 'openmoji_image_path', 'twitter_image_path']] = df_images['hexcode'].progress_apply(lambda x: pd.Series(get_image_paths(x)))

100%|██████████| 4298/4298 [00:54<00:00, 79.29it/s]


In [43]:
df_images

,emoji,hexcode,openmoji_annotation,openmoji_tags_1,openmoji_tags_2,emojipedia_description,llm_tags,llm_description,cleaned_annotations,cleaned_llm_tags,merged_tags,final_tags,tags_embedding,emojipedia_embedding,llm_embedding,combined_embedding,google_image_path,joypixels_image_path,openmoji_image_path,twitter_image_path
0,#️⃣,0023-20e3,keycap: #,keycap,NaN,NaN,NaN,NaN,"keycap, #",,"keycap, #, keycap","#, keycap","[0.0251947, -0.049892798, 0.049407776, -0.0125...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.025194700807332993, -0.04989279806613922, 0...",../data/tensor_images/GoogleEmoji\0023-20e3.pt,../data/tensor_images/JoyPixelsEmoji\0023-20e3.pt,../data/tensor_images/OpenMojiEmoji\0023-20e3.pt,None
1,*️⃣,002a-20e3,keycap: *,keycap,NaN,NaN,NaN,NaN,"keycap, *",,"keycap, *, keycap","*, keycap","[0.030324776, -0.047731027, 0.04585725, -0.008...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.030324775725603104, -0.047731027007102966, ...",../data/tensor_images/GoogleEmoji\002a-20e3.pt,../data/tensor_images/JoyPixelsEmoji\002a-20e3.pt,../data/tensor_images/OpenMojiEmoji\002a-20e3.pt,None
2,-,002d,hyphen-minus,NaN,"hyphen, minus, dash, line",NaN,NaN,NaN,hyphenminus,,"hyphenminus, hyphen, minus, dash, line","dash, hyphen, hyphenminus, line, minus","[-0.024587186, 0.10079277, 0.011220684, -0.040...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.02458718605339527, 0.1007927730679512, 0.0...",None,None,../data/tensor_images/OpenMojiEmoji\002d.pt,None
3,0️⃣,0030-20e3,keycap: 0,keycap,NaN,NaN,NaN,NaN,"keycap, 0",,"keycap, 0, keycap","0, keycap","[0.043693528, -0.016325166, -0.030014966, 0.02...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.043693527579307556, -0.016325166448950768, ...",../data/tensor_images/GoogleEmoji\0030-20e3.pt,../data/tensor_images/JoyPixelsEmoji\0030-20e3.pt,../data/tensor_images/OpenMojiEmoji\0030-20e3.pt,None
4,1️⃣,0031-20e3,keycap: 1,keycap,NaN,NaN,NaN,NaN,"keycap, 1",,"keycap, 1, keycap","1, keycap","[0.02213429, -0.073349945, -0.005648202, -0.00...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.022134289145469666, -0.07334994524717331, -...",../data/tensor_images/GoogleEmoji\0031-20e3.pt,../data/tensor_images/JoyPixelsEmoji\0031-20e3.pt,../data/tensor_images/OpenMojiEmoji\0031-20e3.pt,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4293,,e380,no handshaking,NaN,"hygiene, agreement, virus, meeting, spread, germs",NaN,NaN,NaN,handshaking,,"handshaking, hygiene, agreement, virus, meetin...","agreement, germs, handshaking, hygiene, meetin...","[0.026375707, 0.066247694, -0.0017016568, -0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.026375707238912582, 0.06624769419431686, -0...",None,None,../data/tensor_images/OpenMojiEmoji\e380.pt,None
4294,,e381,web syndication,NaN,"feed, RSS, atom feed, podcast, subscribe, web ...",NaN,NaN,NaN,"web, syndication",,"web, syndication, feed, RSS, atom feed, podcas...","RSS, atom feed, feed, podcast, subscribe, synd...","[0.025086854, -0.12783328, -0.05092573, -0.034...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.02508685365319252, -0.1278332769870758, -0....",None,None,../data/tensor_images/OpenMojiEmoji\e381.pt,None
4295,,f000,windows,NaN,Microsoft,NaN,NaN,NaN,windows,,"windows, Microsoft","Microsoft, windows","[-0.0010178335, -0.015280875, -0.03865494, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.0010178334778174758, -0.015280875377357006...",None,None,../data/tensor_images/OpenMojiEmoji\f000.pt,None
4296,,f77a,artstation,N

In [44]:
df_images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4298 entries, 0 to 4297
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   emoji                   4282 non-null   object
 1   hexcode                 4298 non-null   object
 2   openmoji_annotation     4282 non-null   object
 3   openmoji_tags_1         1906 non-null   object
 4   openmoji_tags_2         391 non-null    object
 5   emojipedia_description  1885 non-null   object
 6   llm_tags                2622 non-null   object
 7   llm_description         2622 non-null   object
 8   cleaned_annotations     4298 non-null   object
 9   cleaned_llm_tags        4298 non-null   object
 10  merged_tags             4298 non-null   object
 11  final_tags              4298 non-null   object
 12  tags_embedding          4298 non-null   object
 13  emojipedia_embedding    4298 non-null   object
 14  llm_embedding           4298 non-null   object
 15  comb

In [45]:
"""
Reference: Chat-GPT-4o
Prompt: 
 
This is my final_df.


#   Column

---  ------

 0   hexcode

 1  combined_embedding



I need to get image path of a specific emoji and add it to the dataframe in a new column "image_path". My images are in the path "../data/tensor_images/". I have emojis of 4 different brands in their respective folders inside the image path ["GoogleEmoji", "JoyPixelsEmoji", "OpenMojiEmoji", "TwitterEmoji"]. I want a new row for each image. Some brands have emoji specific to the hex code, and some do not."""
import os
import pandas as pd
from tqdm import tqdm

# Define base image path and brands
image_base_path = "../data/tensor_images/"
brands = ["GoogleEmoji", "JoyPixelsEmoji", "OpenMojiEmoji", "TwitterEmoji"]

# Function to find all available image paths for a given hexcode
def get_image_paths(hexcode):
    image_paths = {}
   
    for brand in brands:
        brand_path = os.path.join(image_base_path, brand)
        if not os.path.exists(brand_path): # Skip if folder doesn't exist
            continue
           
        expected_filename = f"{hexcode}.pt"  # Adjust based on actual format
       
        if expected_filename in os.listdir(brand_path):
            image_paths[brand] = os.path.join(brand_path, expected_filename)

    return image_paths

# Expand dataframe with tqdm progress bar
expanded_rows = []
for _, row in tqdm(final_df.iterrows(), total=len(final_df), desc="Processing Hexcodes"):
    hexcode = row["hexcode"]
    embedding = row["combined_embedding"]

    image_paths = get_image_paths(hexcode)  # Get list of image paths
   
    if image_paths:  # If images exist, create multiple rows
        for brand, path in image_paths.items():
            expanded_rows.append({"hexcode": hexcode, "combined_embedding": embedding, "image_path": path})
    else:
        # If no images exist, optionally add a row with NaN for image_path
        expanded_rows.append({"hexcode": hexcode, "combined_embedding": embedding, "image_path": None})

# Convert to DataFrame
expanded_df = pd.DataFrame(expanded_rows)

# Optional: Drop rows where no image is found
expanded_df = expanded_df.dropna(subset=["image_path"]).reset_index(drop=True)

Processing Hexcodes: 100%|██████████| 4298/4298 [00:52<00:00, 81.31it/s]


In [46]:
expanded_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12510 entries, 0 to 12509
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   hexcode             12510 non-null  object
 1   combined_embedding  12510 non-null  object
 2   image_path          12510 non-null  object
dtypes: object(3)
memory usage: 293.3+ KB
